In [ ]:
import pandas as pd
import nltk
import numpy as np
import re

#load inthe NTLK stopwords to remove articles, preposition and other words that are not actionable
from nltk.corpus import stopwords
# This allows to create individual objects from a bog of words
from nltk.tokenize import word_tokenize, sent_tokenize
# Lemmatizer helps to reduce words to the base form
from nltk.stem import WordNetLemmatizer

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import os
os.chdir('/content/gdrive/My Drive/Deep Learning/Code')

import pandas as pd
import numpy as np
df_pickle = pd.read_pickle('Final_processed.pkl')

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
df_pickle.info()

In [ ]:
df=pd.read_csv('summer-products-with-rating-and-performance_2020-08.csv')

In [ ]:
df.info()

In [ ]:
index_list = [0, 4, 10, 22, 24, 37, 43, 50, 52, 53, 62, 77, 78, 85, 96, 97, 100, 111, 116, 134, 146, 148, 159, 165, 172, 176, 177, 188, 191, 197, 208, 210, 213, 232, 244, 246, 249, 251, 252, 255, 288, 291, 292, 298, 302, 305, 306, 310, 318, 324, 327, 329, 331, 341, 347, 350, 362, 375, 387, 388, 393, 397, 398, 411, 438, 439, 456, 467, 479, 484, 494, 495, 518, 519, 523, 538, 540, 548, 549, 550, 557, 558, 574, 590, 614, 615, 625, 635, 636, 637, 644, 647, 652, 659, 666, 673, 687, 689, 696, 699, 703, 709, 716, 739, 743, 750, 755, 757, 763, 783, 785, 789, 801, 816, 817, 826, 829, 836, 840, 844, 858, 868, 871, 884, 891, 892, 894, 902, 906, 913, 916, 923, 935, 937, 938, 945, 950, 953, 958, 970, 977, 979, 980, 982, 986, 991, 993, 995, 1003, 1008, 1017, 1026, 1044, 1065, 1069, 1073, 1078, 1083, 1086, 1096, 1108, 1115, 1124, 1129, 1130, 1138, 1143, 1152, 1182, 1186, 1209, 1228, 1230, 1248, 1250, 1261, 1271, 1276, 1279, 1283, 1294, 1311, 1312, 1323, 1326, 1328, 1330, 1343, 1352, 1353, 1361, 1362, 1366, 1368, 1371, 1374, 1380, 1384, 1386, 1390, 1395, 1403, 1417, 1419, 1422, 1428, 1429, 1435, 1437, 1441, 1444, 1456, 1460, 1464, 1471, 1472, 1474, 1477, 1480, 1488, 1491, 1494, 1495, 1516, 1523, 1537, 1541, 1542, 1558, 1559, 1560, 1564, 1568, 1569, 1570]
df = df.drop(index_list)

# Reset the index after dropping rows
df = df.reset_index(drop=True)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# Concatenate the DataFrames horizontally
df = pd.concat([df, df_pickle], axis=1)

In [ ]:
df. info()

In [ ]:
# Plotting with Seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df, x='high_sale', palette=['blue', 'orange'])
plt.xlabel('High Sales')
plt.ylabel('Count')
plt.title('Count of High Sales (0s and 1s)')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import resample

def balance_dataset(df, image_column='image_pixel'):
    # Separate image data and other features
    X_images = np.stack(df[image_column].values)
    y = df['high_sale'].values

    # Balance images using augmentation
    X_images_balanced, y_balanced = augment_minority(X_images, y)

    # Create temporary dataframe without image column
    df_temp = df.drop(columns=[image_column])

    # Separate majority and minority classes for other features
    df_majority = df_temp[df_temp['high_sale'] == 1]
    df_minority = df_temp[df_temp['high_sale'] == 0]

    # Upsample minority class
    df_minority_upsampled = resample(df_minority,
                                   replace=True,
                                   n_samples=len(df_majority),
                                   random_state=42)

    # Combine majority and upsampled minority
    df_numeric_balanced = pd.concat([df_majority, df_minority_upsampled])

    # Create new balanced DataFrame
    df_final = df_numeric_balanced.copy()

    # Add balanced image data
    df_final[image_column] = [img for img in X_images_balanced]

    # Shuffle the dataset
    df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)

    return df_final

def augment_minority(X, y):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Find minority class
    minority_class = 0 if np.sum(y == 0) < np.sum(y == 1) else 1
    majority_class = 1 - minority_class

    # Get minority samples
    X_minority = X[y == minority_class]
    y_minority = y[y == minority_class]

    # Calculate samples needed
    n_minority = len(X_minority)
    n_majority = np.sum(y == majority_class)
    n_to_generate = n_majority - n_minority

    # Generate augmented images
    X_augmented = []
    for i in range(n_to_generate):
        idx = i % n_minority
        img = X_minority[idx:idx+1]
        augmented = next(datagen.flow(img, batch_size=1))[0]
        X_augmented.append(augmented)

    # Combine original and augmented data
    X_balanced = np.concatenate([X, np.array(X_augmented)])
    y_balanced = np.concatenate([y, np.full(n_to_generate, minority_class)])

    return X_balanced, y_balanced

print("Original dataset shape:", df['high_sale'].value_counts())

# Balance the dataset
df_balanced = balance_dataset(df)

print("\nBalanced dataset shape:", df_balanced['high_sale'].value_counts())

# Verify image shapes
print("\nImage shapes in balanced dataset:")
print("First image shape:", np.stack(df_balanced['image_pixel'].iloc[0]).shape)

In [ ]:
df_balanced = balance_dataset(df, image_column='image_pixel')

# Verify the balance
print("Class distribution:", df_balanced['high_sale'].value_counts())

# Access balanced data
X_images = np.stack(df_balanced['image_pixel'].values)
X_numeric = df_balanced.drop(columns=['image_pixel', 'high_sale']).values
y = df_balanced['high_sale'].values

In [ ]:
# Plotting with Seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data=df_balanced, x='high_sale', palette=['blue', 'orange'])
plt.xlabel('High Sales')
plt.ylabel('Count')
plt.title('Count of High Sales (0s and 1s)')
plt.show()

In [ ]:
df_balanced.head()

In [ ]:
df_balanced.info()

In [ ]:
# product color
def main_color(s):
    main_color = {"red":"red", "white":"white", "pink":"pink", "yellow":"yellow", "green":"green", "blue":"blue", "wine":"red", "burgundy":"red", "black":"black", "navy":"navy", "orange":"orange",
    "rose":"pink", "gray":"gray", "grey":"gray", "purple":"purple", "violet":"purple", "army":"green", "leopard":"orange", "ivory":"white",
    "brown":"brown", "coffee":"brown", "camel":"beige", "tan":"brown", "nude":"beige", "khaki":"khaki", "apricot":"yellow", "camouflage":"green", "jasper":"red"}  # ordered by importance
    for key, value in main_color.items():
        if key in s:
            return value
    return "others"
product_color = df_balanced["product_color"]
product_color = [x.lower() if type(x) is str else 'nan' for x in product_color]
product_color = [main_color(color) for color in product_color]
from matplotlib import colors
rgb = []
for s in product_color:
  if s != "others":
    rgb.append(np.array(colors.to_rgb(s)))
rgb = np.array(rgb)
avg_rgb = np.average(rgb, axis=0)
product_colors = [avg_rgb if s == "others" else np.array(colors.to_rgb(s)) for s in product_color]
df_balanced['product_color_rgb'] = product_colors

# log prices
df_balanced['log_price'] = [np.log(p) for p in df_balanced["price"]]
df_balanced['log_retail_price'] = [np.log(p) for p in df_balanced["retail_price"]]

# log merchant rating count
df_balanced['log_merchant_rating_count'] = np.log(df_balanced['merchant_rating_count'])

# urgent text
df_balanced['urgent'] = [1 if s == "Quantité limitée !" else 0 for s in df_balanced["urgency_text"]]
df_balanced["discount_ratio"] = [df_balanced["price"][i]/df_balanced["retail_price"][i] for i in range(len(df_balanced["price"]))]

df_balanced.head()

In [ ]:
rgb = df_balanced["product_color_rgb"]
rgb = np.stack(rgb.values, axis=0)
for i in range(3):
    df_balanced["product_color_rgb"+str(i)] = rgb[:,i]
df_balanced.head()

In [ ]:
def process_sentence(sentence):
    new_tokens = word_tokenize(sentence)
    new_tokens = [token.lower() for token in new_tokens]
    new_tokens =[token for token in new_tokens if token not in stopwords.words('english')]
    new_tokens = [token for token in new_tokens if token.isalpha()]
    lemmatizer = WordNetLemmatizer()
    new_tokens =[lemmatizer.lemmatize(token) for token in new_tokens]
    return "".join(new_tokens)

In [ ]:
titles = df_balanced['title_orig'].tolist()
tokens = [process_sentence(t) for t in titles]
df_balanced['title_preprocessed'] = tokens
df_balanced.head()

In [ ]:
from transformers import BertTokenizer, TFBertModel
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [ ]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import numpy as np
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', from_pt=True)
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

def encode_texts(texts, max_length=128):
    inputs = tokenizer(
        texts.tolist(),
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors="tf"
    )
    return inputs

def get_bert_embeddings(encoded_inputs):
    outputs = bert_model(encoded_inputs['input_ids'],
                        attention_mask=encoded_inputs['attention_mask'])
    return outputs.last_hidden_state[:, 0, :]

def process_in_batches(df, batch_size=32):
    """Process text data in batches to avoid memory issues"""
    all_embeddings = []

    # Calculate number of batches
    n_samples = len(df_balanced)
    n_batches = (n_samples + batch_size - 1) // batch_size

    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, n_samples)

        # Get batch of texts
        batch_texts = df_balanced['title_preprocessed'].iloc[start_idx:end_idx]

        # Process batch
        batch_encodings = encode_texts(batch_texts)
        batch_embeddings = get_bert_embeddings(batch_encodings)

        # Convert to numpy and store
        all_embeddings.append(batch_embeddings.numpy())

        # Optional: Print progress
        print(f"Processed batch {i+1}/{n_batches}", end='\r')

    # Combine all batches
    final_embeddings = np.vstack(all_embeddings)
    print("\nProcessing complete!")
    return final_embeddings

# Process data in batches
bert_embeddings_np = process_in_batches(df_balanced, batch_size=32)

print("BERT embeddings shape:", bert_embeddings_np.shape)

# Add embeddings as features
for i in range(bert_embeddings_np.shape[1]):
    df_balanced[f'bert_feature_{i}'] = bert_embeddings_np[:, i]

In [ ]:
df_balanced.info()

In [ ]:
X = df_balanced[["log_price", "log_retail_price", "uses_ad_boosts", "badges_count", "badge_local_product","badge_product_quality", "badge_fast_shipping", "urgent", "discount_ratio",
                    "product_color_rgb0", "product_color_rgb1", "product_color_rgb2", "shipping_option_price", "shipping_is_express", "countries_shipped_to", "inventory_total", "image_pixel"]]
y = df_balanced['high_sale']

In [ ]:
# Loop over column names and assign them to X
for i in range(768):  # Since there are 768 BERT features
    X[f'bert_feature_{i}'] = df_balanced[f'bert_feature_{i}']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
def create_combined_model(input_shape_image, input_shape_bert, input_shape_numeric):
    # Input layers
    image_input = Input(shape=input_shape_image, name='image_input')
    bert_input = Input(shape=(input_shape_bert,), name='bert_input')
    numeric_input = Input(shape=(input_shape_numeric,), name='numeric_input')

    # Image processing branch - increased capacity
    base_model = EfficientNetB5(weights='imagenet',
                               include_top=False,
                               input_shape=input_shape_image)

    # Freeze fewer layers (70%) to allow more learning
    for layer in base_model.layers[:int(0.7 * len(base_model.layers))]:
        layer.trainable = False

    image_x = base_model(image_input)
    image_x = GlobalAveragePooling2D()(image_x)
    image_x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(image_x)
    image_x = BatchNormalization()(image_x)
    image_x = Dropout(0.3)(image_x)

    # BERT features processing - deeper network
    bert_x = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(bert_input)
    bert_x = BatchNormalization()(bert_x)
    bert_x = Dropout(0.3)(bert_x)

    bert_x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(bert_x)
    bert_x = BatchNormalization()(bert_x)
    bert_x = Dropout(0.2)(bert_x)

    # Numeric features processing - enhanced
    numeric_x = BatchNormalization()(numeric_input)
    numeric_x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(numeric_x)
    numeric_x = BatchNormalization()(numeric_x)
    numeric_x = Dropout(0.2)(numeric_x)

    # Combine features with attention mechanism
    combined = Concatenate()([image_x, bert_x, numeric_x])

    # Deeper fusion layers
    x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(combined)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    x = Dense(128, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)

    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[image_input, bert_input, numeric_input], outputs=output)

    optimizer = Adam(learning_rate=1e-4)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB5
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import tensorflow as tf
from keras import Input
# Prepare data
numeric_features = ['log_price', 'log_retail_price', 'uses_ad_boosts', 'badges_count',
                   'badge_local_product', 'badge_product_quality', 'badge_fast_shipping',
                   'urgent', 'discount_ratio', 'product_color_rgb0', 'product_color_rgb1',
                   'product_color_rgb2', 'shipping_option_price', 'shipping_is_express',
                   'countries_shipped_to', 'inventory_total']
bert_features = [f'bert_feature_{i}' for i in range(768)]

# Prepare all data types
X_numeric = df_balanced[numeric_features].values
X_bert = df_balanced[bert_features].values
X_image = np.stack(df_balanced['image_pixel'].values)
y = df_balanced['high_sale'].values

# Normalize image data
X_image = X_image / 255.0

# Split data maintaining alignment
X_image_train, X_image_test, X_bert_train, X_bert_test, X_numeric_train, X_numeric_test, y_train, y_test = train_test_split(
    X_image, X_bert, X_numeric, y, test_size=0.125, random_state=42
)

X_image_train, X_image_val, X_bert_train, X_bert_val, X_numeric_train, X_numeric_val, y_train, y_val = train_test_split(
    X_image_train, X_bert_train, X_numeric_train, y_train, test_size=0.14286, random_state=42
)

# Create and train model
input_shape_image = X_image.shape[1:]
model = create_combined_model(
    input_shape_image=input_shape_image,
    input_shape_bert=768,
    input_shape_numeric=len(numeric_features)
)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        min_delta=0.001
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weight_dict = dict(enumerate(class_weights))

# Training with modified parameters
history = model.fit(
    [X_image_train, X_bert_train, X_numeric_train],
    y_train,
    validation_data=([X_image_val, X_bert_val, X_numeric_val], y_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    class_weight=class_weight_dict,
    verbose=1
)

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np

# Predict on the test data
y_pred_prob = model.predict([X_image_test, X_bert_test, X_numeric_test])

# Convert probabilities to binary predictions (threshold 0.5 for sigmoid activation)
y_pred = (y_pred_prob >= 0.5).astype(int).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test data: {accuracy:.2f}")
